In [27]:
import torch
import torch.nn as nn
import torch.optim as optim

from model_factories import get_golem_model, get_resnet18_model, get_resnet34_model, get_vitbase_model
from dataloaders import get_dataloaders
from model_utils import train_epoch, eval

In [28]:
train_loader, test_loader, classes = get_dataloaders()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gc = get_resnet18_model()

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(gc.parameters(), lr=1e-3)

In [30]:
for i in range(2):
    loss = train_epoch(gc, criterion, optimizer, train_loader, device)
    print(f"{i+1}/2: loss={loss}")

1/2: loss=2313.923654615879
2/2: loss=1661.978788882494


In [31]:
loss, acc = eval(gc, criterion, test_loader, device)
print(f"Avg loss {loss}\nAccu {acc}")

Avg loss 6721
Accu 10000
